<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span><ul class="toc-item"><li><span><a href="#Промежуточные-выводы" data-toc-modified-id="Промежуточные-выводы-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Промежуточные выводы</a></span></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. 

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

# Инструкция по выполнению проекта

1. Загрузка и подготовка данных
2. Исследование баланса классов, обучение модели без учёта дисбаланса.
3. Улучшение качества модели, учитывая дисбаланс классов. 
4. Тестирование модели.


# Описание данных.     

***Признаки***     
- `RowNumber` — индекс строки в данных     
- `CustomerId` — уникальный идентификатор клиента     
- `Surname` — фамилия     
- `CreditScore` — кредитный рейтинг     
- `Geography` — страна проживания     
- `Gender` — пол     
- `Age` — возраст     
- `Tenure` — сколько лет человек является клиентом банка     
- `Balance` — баланс на счёте     
- `NumOfProducts` — количество продуктов банка, используемых клиентом     
- `HasCrCard` — наличие кредитной карты     
- `IsActiveMember` — активность клиента     
- `EstimatedSalary` — предполагаемая зарплата     
***Целевой признак***     
- `Exited` — факт ухода клиента

## Подготовка данных

Импортируем необходимые библиотеки

In [1]:
#импортируем библиотеки pandas и matplotlib
import pandas as pd

import matplotlib.pyplot as plt 

In [22]:
# из библиотеки sklearn импортируем необходимые инструменты
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV #добавим работу с grid search для поиска гипермараметров модели
from sklearn.model_selection import cross_validate #добавим работу с кросс-валидацией

<div class="alert alert-block alert-success">
<b>Успех:</b> Импорты на месте
</div>


In [3]:
#откроем файл с данными
try:
    df = pd.read_csv('/datasets/Churn.csv')
except:
    df = pd.read_csv('/Users/moygospodin/Documents/Practicum/Projects/Iskandarov_customer_churn/datasets/Churn.csv')


In [4]:
#посмотрим на часть данных для поверхностного изучения
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [5]:
#удалим столбец `RowNumber`, так как он не несет смысловой нагрузки
df = df.drop(['RowNumber', 'CustomerId'], axis=1)

In [6]:
#изучим датасет на пропуски в данных
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Surname          10000 non-null  object 
 1   CreditScore      10000 non-null  int64  
 2   Geography        10000 non-null  object 
 3   Gender           10000 non-null  object 
 4   Age              10000 non-null  int64  
 5   Tenure           9091 non-null   float64
 6   Balance          10000 non-null  float64
 7   NumOfProducts    10000 non-null  int64  
 8   HasCrCard        10000 non-null  int64  
 9   IsActiveMember   10000 non-null  int64  
 10  EstimatedSalary  10000 non-null  float64
 11  Exited           10000 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 937.6+ KB


Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

In [7]:
#посмотрим, какие значения имеются в столбце
df['Tenure'].value_counts()

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: Tenure, dtype: int64

В столбце `Tenure` содержится информация о том, сколько лет человек является клиентом в банке.    
Очевидно, что внесение значений в виде медианы не подойдет.     
Заполнить пропуски не удастся, поэтому строки с пропущенными значениями оптимально удалить, так как они могут повлиять на обучение модели.

In [8]:
df = df.dropna()

Почти закончили с предобработкой.     
С помощью порядкового кодирования переведем категориальные признаки в числовые, далее подготовим выборки для обучения, валидации и тестирования.

In [9]:
data = pd.DataFrame(OrdinalEncoder().fit_transform(df),
                            columns=df.columns)

In [10]:
features = data.drop(['Exited'], axis=1)
target = data['Exited']

In [11]:
#делим на выборки для обучения и валидации+тестирования
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345, stratify=target
)

In [12]:
#делим выборку на валидацию и тестирование
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345, stratify=target_valid
)

In [13]:
#проверим, что разделение на выборки произошло, как мы планировали
display(features_train.shape[0], features_valid.shape[0], features_test.shape[0])

6818

1136

1137

## Исследование задачи

Для начала посмотрим насколько сбалансированны классы в датафрейме, чтобы понять с чем нам придется иметь дело.     

После этого, напишем функции, которые с помощью цикла найдут лучшие гиперпараметры для нашей модели.

In [14]:
#посмотрим на сбалансированность классов
target.value_counts()

0.0    7237
1.0    1854
Name: Exited, dtype: int64

Как можно заметить, у нас отрицательный класс 3.9 раз превосходит положительный.     
На данном этапе мы ничего с этим делать не будем, посмотрим как модель справить с этими входными признаками.

Найдем гиперпараметры модели с помощью GridSearchCV

In [45]:
%%time
model_RFC = RandomForestClassifier(class_weight='balanced', random_state=42)
param_grid = {
    'criterion': ('gini', 'entropy'),
    'max_depth': (1, 11, 5),
    'n_estimators': range (10, 51, 10),
    'min_samples_leaf': range (2, 11, 2),
    'min_samples_split': range (2, 11 ,2) 
}
gs_RFC = GridSearchCV(model_RFC, param_grid=param_grid,
                      scoring='f1', n_jobs=-1, cv=3)
gs_RFC.fit(features_train, target_train)

CPU times: user 2.87 s, sys: 310 ms, total: 3.17 s
Wall time: 1min 40s


GridSearchCV(cv=3,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=42),
             n_jobs=-1,
             param_grid={'criterion': ('gini', 'entropy'),
                         'max_depth': (1, 11, 5),
                         'min_samples_leaf': range(2, 11, 2),
                         'min_samples_split': range(2, 11, 2),
                         'n_estimators': range(10, 51, 10)},
             scoring='f1')

Передадим гипермараметры и напишем функцию для кросс-валидационной проверки модели

In [46]:
best_RFC = gs_RFC.best_estimator_

In [47]:
def cross_valid(model):
    scores = cross_validate(model, features_train, target_train, cv=3,
                            scoring=['f1', 'roc_auc', 'accuracy'],
                            return_train_score=True)

    print(f"Усредненный показатель F1 меры на модели {scores['test_f1'].mean()}")
    print(f"Усредненный показатель AUC-ROC на модели {scores['test_roc_auc'].mean()}")
    print(f"Усредненный показатель Accuracy на модели {scores['test_accuracy'].mean()}")    
    

In [48]:
cross_valid(best_RFC)
display(best_RFC)

Усредненный показатель F1 меры на модели 0.6121682705938302
Усредненный показатель AUC-ROC на модели 0.8540783076632898
Усредненный показатель Accuracy на модели 0.8417442254347319


RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=11, min_samples_leaf=4, min_samples_split=10,
                       n_estimators=30, random_state=42)


Как результат кросс-валидации имеем случайный лес со следующими гиперпараметрами:
- Глубина: 11
- Количество деревьев: 30     

Данная модель показала следующие результаты на валидационной выборке:
1. F1-мера: 61.2%
2. AUC-ROC: 85.4%

Как можно заметить по AUC-ROC - наша модель далеко ушла от модели, которая делает случайные прогнозы.     
F1-мера также имеет неплохие показатели, но поскольку мы работаем с данными, которые имеют довольно серьезный дисбаланс - посмотрим как она изменится после сэмплирования признаков.

## Борьба с дисбалансом

В предыдущем шаге мы уже смотрели на то, какой имеется дисбаланс. Продублируем:

- 0.0  -  7237
- 1.0  -  1854

Имеем практически 4-х кратную разницу.     

В этом шаге мы сделаем следующее:
1. Посмотрим как поведет себя модель, когда осуществлен Upsampling
2. Посмотрим на поведение модели, когда осуществлен Downsampling
3. Выберем модель с наилучшими показателями F1-меры

In [25]:
#напишем функцию для апсэмплинга данных
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345
    )
    return features_upsampled, target_upsampled

In [26]:
#применим функцию к обучающим выборкам
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

In [27]:
#проверим получившийся результат
target_upsampled.value_counts()

1.0    5560
0.0    5428
Name: Exited, dtype: int64

In [43]:
%%time
model_RFC = RandomForestClassifier(class_weight='balanced', random_state=42)
param_grid = {
    'criterion': ('gini', 'entropy'),
    'max_depth': (1, 11, 5),
    'n_estimators': range (10, 51, 10),
    'min_samples_leaf': range (2, 11, 2),
    'min_samples_split': range (2, 11 ,2) 
}
gs_RFC = GridSearchCV(model_RFC, param_grid=param_grid,
                      scoring='f1', n_jobs=-1, cv=3)
gs_RFC.fit(features_upsampled, target_upsampled)

best_RFC_upsampled = gs_RFC.best_estimator_

CPU times: user 3.56 s, sys: 412 ms, total: 3.97 s
Wall time: 2min 21s


In [44]:
cross_valid(best_RFC_upsampled)
display(best_RFC_upsampled)

Усредненный показатель F1 меры на модели 0.5928454530600146
Усредненный показатель AUC-ROC на модели 0.8513459792451114
Усредненный показатель Accuracy на модели 0.842623732569932


RandomForestClassifier(class_weight='balanced', max_depth=11,
                       min_samples_leaf=2, n_estimators=50, random_state=42)

Как можно заметить, после апсемплинга у нас упал показатель f1-меры.   

Данная модель показала следующие результаты на валидационной выборке:
1. F1-мера: 59.3% 
2. AUC-ROC: 85.1% 

Далее изучим поведение модели при даунсэмплинге.

In [34]:
#напишем функцию для даунсэмлинга
def downsample(features, target, fraction):
    features_zeros = features[target == 0] 
    features_ones = features[target == 1] 
    target_zeros = target[target == 0] 
    target_ones = target[target == 1]
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones]
    )
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones]
    )
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345
    )
    return features_downsampled, target_downsampled

In [35]:
#применим ее на обучающей выборке
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

In [36]:
#проверим результат ее работы
target_downsampled.value_counts()

1.0    1390
0.0    1357
Name: Exited, dtype: int64

In [41]:
%%time
model_RFC = RandomForestClassifier(class_weight='balanced', random_state=42)
param_grid = {
    'criterion': ('gini', 'entropy'),
    'max_depth': (1, 11, 5),
    'n_estimators': range (10, 51, 10),
    'min_samples_leaf': range (2, 11, 2),
    'min_samples_split': range (2, 11 ,2) 
}
gs_RFC = GridSearchCV(model_RFC, param_grid=param_grid,
                      scoring='f1', n_jobs=-1, cv=3)
gs_RFC.fit(features_downsampled, target_downsampled)

best_RFC_downsampled = gs_RFC.best_estimator_

CPU times: user 2.66 s, sys: 234 ms, total: 2.89 s
Wall time: 1min 1s


In [42]:
cross_valid(best_RFC_downsampled)
display(best_RFC_downsampled)

Усредненный показатель F1 меры на модели 0.6073042217102633
Усредненный показатель AUC-ROC на модели 0.855713907835352
Усредненный показатель Accuracy на модели 0.8410102700821441


RandomForestClassifier(class_weight='balanced', max_depth=11,
                       min_samples_leaf=4, n_estimators=50, random_state=42)


Данная модель после даунсемплинга показала следующие результаты на валидационной выборке:
1. F1-мера: 60.7%
2. AUC-ROC: 85.6%

Эти показатели также ниже, чем у модели без измененных данных для обучения

### Промежуточные выводы

Наилучшая метрика F1-меры у модели, которая была обучена на данных через Upsampling.

Данная модель имеет следующие гиперпараметры:
- Глубина: 16
- Количество деревьев: 101

Случайный лес с такими гиперпараметрами мы и будем проверять на тестовой выборке.

## Тестирование модели

Проведем тестирование случайного леса на тестовой выборке.

In [49]:
model = best_RFC

predictions_valid = model.predict(features_test)
f1 = f1_score(target_test, predictions_valid)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_valid)

print(f1)
print(auc_roc)

0.6242299794661191
0.8680653457801486


### Вывод:

Случайный лес с гиперпараметрами:
- глубина 11
- количество деревьев 30

Который был обучен на данных после устранения дисбаланса с помощью апсэмплинга имеет следующие метрики:
F1-мера: ***62.4%*** 
AUC-ROC: ***86.8%***


Семплирование синтетическими данными, как и их уменьшение для борьбы с дисбалансом видимых результатов не дало.